In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
Rx = PolynomialRing(QQ, 'x')
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
Scplx= PowerSeriesRing(CC,'x',default_prec=300)

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

def interpolate(data):
    rl=R.lagrange_polynomial(data)
    rl=D(rl)
    return rl

def divides(a,b):
    if mod(b,a)==0:return True
    return False

def order(p,n): #seems to be the fastest way
    lst=n.digits(p)
    count=0
    k=0
    while lst[k]==0:
        count=count+1
        k=k+1
    return count

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant



def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def H(n,m): # definition 8.1 (13aug21)
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    power = 1/(m-2)
    frac=numerator/denominator
    return ((S(frac^power)).O(n+1)).polynomial()

def H4(n,m): #per dfmn 8.2 in 2021 Integer paper
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
            
def deltaInftyStrike(n,m): 
    # substitute 2^6*m^3*x for x in def 10.1 of interpolations paper
    f=deltaInfty(n,m)
    fl=f.list()
    df=f.degree()
    sm=sum(fl[k]*(x*m^3*2^6)^k for k in [0..df])
    dsm=sm.degree()
    sml=sm.list()
    notzero=[]
    for entry in sml:
        if (entry!=0):notzero=notzero+[entry]
    leadingterm=notzero[0]
    sm2=sum(fl[k]*(x*m^3*2^6)^k/leadingterm for k in [0..dsm])
    return sm2

def deltaInfty(n,m): 
    # definition 10.1 in the paper
    # so, per remark 4 in final version,
    # the weight is 4m/(m-2) for each m.
    jay=J(n,m)
    djay=x*derivative(jay,x)
    numerator=djay^(2*m)
    denominator=jay^(2*m-2)*(jay-1)^m
    answer=S((numerator/denominator)^(1/(m-2)))
    return answer.polynomial()

def reciprocalDeltaInfty(n,m):
    return (L(1/deltaInfty(n,m))).O(n)
            
def deltaInftyStrike(n,m): 
    f=deltaInfty(n,m)
    fl=f.list()
    df=f.degree()
    sm=sum(fl[k]*(x*m^3*2^6)^k for k in [0..df])
    dsm=sm.degree()
    sml=sm.list()
    notzero=[]
    for entry in sml:
        if (entry!=0):notzero=notzero+[entry]
    leadingterm=notzero[0]
    sm2=sum(fl[k]*(x*m^3*2^6)^k/leadingterm for k in [0..dsm])
    return sm2

def nieburTau(n):
# The contortions below sidestep mysterious (to me) error messages...
# nieburTau is an algorith for Ramanujan''s tau from the paper
# "A formula for Ramanujan's tau-function" by Douglas Niebur.
# (He spells tau with the Greek letter.)
    def step1(n,k):
        return 35*k^4-52*k^3*n+18*k^2*n^2
    def step2(n,k):
        return sigma(k)*sigma(n-k)
    def step3(n):
        ans=0
        for k in [1..n-1]:
            ans=ans+step1(n,k)*step2(n,k)
        return 24*ans
    return n^4*sigma(n)-step3(n)

def first(list):return list[0]
def last(list):return list[-1]

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

P=Primes()
def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def multiplicative_order_integer(p,n):
    fi=factor_integer(n)
    for pair in fi:
        if pair[0]==p:return pair[1]
    return 0

def multiplicative_order_integerOld(p,n):
    tv=[]
    answers=[0]
    for k in [0..20]:
        old=tv
        tv=tv+[n/p^k in ZZ]
        if (False in tv)&((False in old)==False):answers=answers+[k-1]
    return max(answers)

def multiplicative_order(d,n): 
    n=QQ(n)
    num=ZZ(numerator(n))
    den=ZZ(denominator(n))
    on=multiplicative_order_integer(d,num)
    od=multiplicative_order_integer(d,den)
    return on-od

def multiplicative_orderOld(d,n): #apparently, Old version is faster sometimes.
    n=QQ(n)
    num=ZZ(numerator(n))
    den=ZZ(denominator(n))
    on=multiplicative_order_integerOld(d,num)
    od=multiplicative_order_integerOld(d,den)
    return on-od



def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jNormalizedPowerConstantTerm(m,power,change):
    trun=L(j(power,m)-QQ(change))
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def x_reciprocalDeltaInftyStep(n,m):
    f=reciprocalDeltaInfty(n,m)
    fl=f.list()
    rt=0
    for i in [0..len(fl)-1]:
        rt=rt+fl[i]*x**i*(m**3*2**6)**i
    return rt
    
def x_reciprocalDeltaInfty(n,m):return x_reciprocalDeltaInftyStep(n,m)

def reciprocalDeltaInfty_Strike(n,m):
    l_srs=L(x_reciprocalDeltaInfty(n,m))/x
    return l_srs

def reciprocalDeltaInfty_StrikeConstantTerm(m,power):
    trun=reciprocalDeltaInfty_Strike(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [ ]:
# output has been snipped. 
# This routine was executed in 'reciprocalDeltaInftyStrike30oct22.ipynb'
polydata=[]
numericalfactors=[]
constants=[]
import time
for power in [1..50]:
    start=time.time()/60
    data=[]
    for m in [3..3*power+3]:
        cst=reciprocalDeltaInfty_StrikeConstantTerm(m,power)
        data=data+[(m,cst)]
    rl=R.lagrange_polynomial(data)
    nf=QQ(numericalfactor(rl))
    pair=[power,nf]
    numericalfactors=numericalfactors+[pair]
    pair=[power,rl]
    polydata=polydata+[pair]
    minutes=time.time()/60-start
    print("(power,elapsed minutes): ",(power,minutes))
    print("===============================================================================")
print("numerical factors:")
print(numericalfactors) 
import pickle
wfile = open('/Users/barrybrent/1nov22no1.txt','wb') 
s = pickle.dumps(str(polydata))
wfile.write(s)
wfile.close()
wfile2 = open('/Users/barrybrent/1nov22no2.txt','wb') 
s2 = pickle.dumps(str(numericalfactors))
wfile2.write(s2)
wfile2.close()

In [3]:
import pickle
rfile = open('/Users/barrybrent/1nov22no2.txt','rb')   # above numerical factor file
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
print(s)

[[1, 8], [2, 136], [3, 2624], [4, 53416], [5, 1120448], [6, 23958208], [7, 519193088], [8, 11362852072], [9, 250573558208], [10, 5558899445056], [11, 123924641000960], [12, 2773818448044736], [13, 62297785851098624], [14, 1403218074149034496], [15, 31685737403783450624], [16, 717050363407819089256], [17, 16258064812498394969024], [18, 369255449106583167693376], [19, 8399328746316000272705024], [20, 191317498144666442129440576], [21, 4363147117757259217650696704], [22, 99616351783644180286165904896], [23, 2276691218595921288807130886144], [24, 52081431256395763400795768871616], [25, 1192433257951026153362353593179648], [26, 27323013848628811873415148818841088], [27, 626528320143835261586566889650270208], [28, 14376309715585849541399638882908815872], [29, 330086814481532726562536451938024910848], [30, 7583409166141615008658810560391364620288], [31, 174316975559471079793395123448068081483776], [32, 4009024038223489556479143243536203509692008], [33, 9224603655896379729402036788859424950674

In [7]:
import pickle
rfile = open('/Users/barrybrent/1nov22no2.txt','rb')   # above numerical factor file
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
diffs=[]
for k in [0..49]:
    power=s[k][0]
    dsp=digitsum(2,power)
    nf=s[k][1]
    o2=order(2,nf)
    diff=o2-3*dsp
    diffs=diffs+[diff]
print(diffs)
    

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [3]:
import pickle
rfile = open('/Users/barrybrent/1nov22no2.txt','rb')   # above numerical factor file
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]

for k in [0..49]:
    power=s[k][0]
    dsp=digitsum(3,power)
    nf=s[k][1]
    o3=order(3,nf)
    data=data+[o3]
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [15]:
rfile = open('/Users/barrybrent/1nov22no2.txt','rb')   # above numerical factor file
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..49]:
    power=s[k][0]
    nf=s[k][1]
    data=data+[nf]
print(data)

[8, 136, 2624, 53416, 1120448, 23958208, 519193088, 11362852072, 250573558208, 5558899445056, 123924641000960, 2773818448044736, 62297785851098624, 1403218074149034496, 31685737403783450624, 717050363407819089256, 16258064812498394969024, 369255449106583167693376, 8399328746316000272705024, 191317498144666442129440576, 4363147117757259217650696704, 99616351783644180286165904896, 2276691218595921288807130886144, 52081431256395763400795768871616, 1192433257951026153362353593179648, 27323013848628811873415148818841088, 626528320143835261586566889650270208, 14376309715585849541399638882908815872, 330086814481532726562536451938024910848, 7583409166141615008658810560391364620288, 174316975559471079793395123448068081483776, 4009024038223489556479143243536203509692008, 92246036558963797294020367888594249506748352, 2123510248900770288342458888150686191436176448, 48904316524311797944877998397777155698865591808, 1126719387108348117847456004006718092292819996736, 2596871689466630582647597867495816

In [16]:
import pickle
rfile = open('/Users/barrybrent/1nov22no2.txt','rb')   # above numerical factor file
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..49]:
    power=s[k][0]
    nf=s[k][1]
    o2=order(2,nf)
    data=data+[nf/2^o2]
print(data)
    

[1, 17, 41, 6677, 17507, 374347, 1014049, 1420356509, 3915211847, 86857803829, 242040314455, 43340913250699, 121675362990427, 2740660301072333, 7735775733345569, 89631295425977386157, 254032262695287421391, 5769616392290361995209, 16404938957648438032627, 2989335908510413158272509, 8521771714369646909474017, 194563187077430039621417783, 555832817040019845900178439, 813772363381183803137433888619, 2328971206935597955785846861679, 53365261423103148190263962536799, 152961015660116030660782932043523, 28078729913253612385546169693181281, 80587601191780450820931750961431863, 1851418253452542726723342421970548003, 5319731920149874261273044538820437057, 501128004777936194559892905442025438711501, 1441344321233809332719068248259285148542943, 33179847639074535755350920127354471741190257, 95516243211546480361089840620658507224346859, 17604990423567939341366500062604970192075312449, 50720150184895128567335895849527669048485782349, 1169425557209551261239646033446657688422085920731, 3371505159259287

In [17]:
import pickle
rfile = open('/Users/barrybrent/1nov22no2.txt','rb')   # above numerical factor file
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..49]:
    power=s[k][0]
    nf=s[k][1]
    o2=order(2,nf)
    print((power,nf/2^o2))

(1, 1)
(2, 17)
(3, 41)
(4, 6677)
(5, 17507)
(6, 374347)
(7, 1014049)
(8, 1420356509)
(9, 3915211847)
(10, 86857803829)
(11, 242040314455)
(12, 43340913250699)
(13, 121675362990427)
(14, 2740660301072333)
(15, 7735775733345569)
(16, 89631295425977386157)
(17, 254032262695287421391)
(18, 5769616392290361995209)
(19, 16404938957648438032627)
(20, 2989335908510413158272509)
(21, 8521771714369646909474017)
(22, 194563187077430039621417783)
(23, 555832817040019845900178439)
(24, 813772363381183803137433888619)
(25, 2328971206935597955785846861679)
(26, 53365261423103148190263962536799)
(27, 152961015660116030660782932043523)
(28, 28078729913253612385546169693181281)
(29, 80587601191780450820931750961431863)
(30, 1851418253452542726723342421970548003)
(31, 5319731920149874261273044538820437057)
(32, 501128004777936194559892905442025438711501)
(33, 1441344321233809332719068248259285148542943)
(34, 33179847639074535755350920127354471741190257)
(35, 95516243211546480361089840620658507224346859)


In [18]:
import pickle
rfile = open('/Users/barrybrent/1nov22no2.txt','rb')   # above numerical factor file
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..49]:
    power=s[k][0]
    nf=s[k][1]
    print((power,nf))

(1, 8)
(2, 136)
(3, 2624)
(4, 53416)
(5, 1120448)
(6, 23958208)
(7, 519193088)
(8, 11362852072)
(9, 250573558208)
(10, 5558899445056)
(11, 123924641000960)
(12, 2773818448044736)
(13, 62297785851098624)
(14, 1403218074149034496)
(15, 31685737403783450624)
(16, 717050363407819089256)
(17, 16258064812498394969024)
(18, 369255449106583167693376)
(19, 8399328746316000272705024)
(20, 191317498144666442129440576)
(21, 4363147117757259217650696704)
(22, 99616351783644180286165904896)
(23, 2276691218595921288807130886144)
(24, 52081431256395763400795768871616)
(25, 1192433257951026153362353593179648)
(26, 27323013848628811873415148818841088)
(27, 626528320143835261586566889650270208)
(28, 14376309715585849541399638882908815872)
(29, 330086814481532726562536451938024910848)
(30, 7583409166141615008658810560391364620288)
(31, 174316975559471079793395123448068081483776)
(32, 4009024038223489556479143243536203509692008)
(33, 92246036558963797294020367888594249506748352)
(34, 2123510248900770288342

In [19]:
import pickle
rfile = open('/Users/barrybrent/1nov22no2.txt','rb')   # above numerical factor file
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..49]:
    power=s[k][0]
    nf=s[k][1]
    data=data+[nf]
print(data)

[8, 136, 2624, 53416, 1120448, 23958208, 519193088, 11362852072, 250573558208, 5558899445056, 123924641000960, 2773818448044736, 62297785851098624, 1403218074149034496, 31685737403783450624, 717050363407819089256, 16258064812498394969024, 369255449106583167693376, 8399328746316000272705024, 191317498144666442129440576, 4363147117757259217650696704, 99616351783644180286165904896, 2276691218595921288807130886144, 52081431256395763400795768871616, 1192433257951026153362353593179648, 27323013848628811873415148818841088, 626528320143835261586566889650270208, 14376309715585849541399638882908815872, 330086814481532726562536451938024910848, 7583409166141615008658810560391364620288, 174316975559471079793395123448068081483776, 4009024038223489556479143243536203509692008, 92246036558963797294020367888594249506748352, 2123510248900770288342458888150686191436176448, 48904316524311797944877998397777155698865591808, 1126719387108348117847456004006718092292819996736, 2596871689466630582647597867495816